### **GloVe**  
임베딩 벡터의 내적이 말뭉치 전체에서의 동시출현(co-occurrence) 확률 값이 되는 목적 함수를 갖는다.  
이를 통해 임베딩 벡터간 유사도 측정을 수월하게 하면서도 말뭉치 전체의 통계 정보를 반영할 수 있다. 

> **동시 출현 (Co-occurence)** *이란, 한 문장, 문단 또는 텍스트 단위에서 같이 출현한 단어를 가리다. 언어학적 의미에서 의미적 근접성을 가리킨다.*  




In [3]:
import os
import re
from pathlib import Path

In [4]:
BASE_DIR = "/data/ksb/TestDir"
DATA_BASE_DIR = os.path.join(BASE_DIR, "sample_articles")

ORIGIN_PATH = os.path.join(DATA_BASE_DIR,"Origin-Data")
PREPROCESSED_PATH = os.path.join(DATA_BASE_DIR,"Preprocessed-Data")
PRETTY_PATH = os.path.join(DATA_BASE_DIR,"Pretty-Data")
SWORDS_PATH = os.path.join(DATA_BASE_DIR, "StopWordList.txt")
MODEL_PATH = os.path.join(os.path.join(Path(os.getcwd()).parent, "Word-Embedding-Model"))

In [5]:
class RawTextReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile("/n")

    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in ch:
                yield s

In [6]:
def mkdir_p(path):
    import errno
    try:
        os.makedirs(path)
    except OSError as exc:
        if exc.errno == errno.EEXIST and os.path.isdir(path):
            pass
        else:
            raise


기사 본문 내용이 짧기 때문에, 모델을 학습하기에 corpus의 크기가 작다.  
아래는 수집한 기사 87건들을 통해 corpus를 구성하고, GloVe 모델을 구축하는 내용이다.

In [8]:
media_list = os.listdir(PREPROCESSED_PATH)

result = []
forCount = []
for media in media_list:
    media_path = os.path.join(PREPROCESSED_PATH, media)
    article_list= os.listdir(media_path)

    for article in article_list:
        reader = RawTextReader(os.path.join(media_path, article)) 
        content = list(filter(None, reader))
        forCount += [token for sent in content for token in sent.split()]
        result += [sent.split() for sent in content]

In [9]:
print("전체 token의 개수 : {len}".format(len=len(forCount)))
print("중복되지 않은 token의 개수 : {len}".format(len=len(list(set(forCount)))))

전체 token의 개수 : 19644153
중복되지 않은 token의 개수 : 66456


In [12]:
START_TOKEN = ['<SOS>']
END_TOKEN = ['<EOS>']

In [13]:
result = list(map(lambda content : START_TOKEN + content + END_TOKEN, result))

In [14]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=5)

In [15]:
glove = Glove(no_components=256, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)
glove.add_dictionary(corpus.dictionary)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


경찰서장과 비슷한 의미의 단어를 출력한다.

In [16]:
model_result1=glove.most_similar("경찰서장")
print(model_result1)

[('양천', 0.7152219849493041), ('최진태', 0.547380345529357), ('해양', 0.5423396482507843), ('백학', 0.541540346122516)]


In [17]:
corpus.dictionary["선거"]

668

In [18]:
glove.word_vectors[corpus.dictionary["선거"]]

array([-5.86857987e-02,  3.04725902e-02, -3.50713933e-01,  1.63890719e-01,
        1.18662185e-01, -3.57819899e-01, -4.54589913e-02,  2.49132047e-01,
        1.41620210e-01,  4.52376384e-01, -2.02558500e-01, -2.55210691e-01,
        6.89365200e-01,  1.91398015e-01, -1.70875454e-01,  2.12077581e-01,
       -2.03151912e-01, -4.13377844e-02,  1.96604925e-01, -1.68379841e-01,
       -2.58124378e-01, -1.81894005e-01,  3.10510050e-01, -2.53778388e-01,
        1.25946503e-01, -1.12744791e-01, -3.08914865e-02,  1.93688787e-01,
        1.51799771e-01, -2.41808754e-01, -1.93119695e-01, -1.00111310e-01,
       -1.53392780e-01, -1.20103478e-01,  2.54215635e-01,  3.44016066e-01,
       -1.76844752e-01,  7.43174397e-02,  1.71243858e-01, -5.69981152e-02,
        6.99238785e-02,  1.32170250e-01, -9.71812092e-02, -4.34700657e-02,
       -1.33847426e-02, -2.24707559e-01, -8.96834802e-02, -1.49370908e-01,
        2.86347221e-01, -3.15668919e-01, -8.34240792e-02, -4.26584337e-01,
       -1.48346810e-01,  

In [19]:
mkdir_p(MODEL_PATH)

corpus_path = os.path.join(MODEL_PATH, "corpus-256.model")
model_path = os.path.join(MODEL_PATH, 'glove-256.model')


glove.save(model_path)
corpus.save(corpus_path)